In [ ]:
%cd /content/PaddleOCR-Vietnamese

/content/PaddleOCR-Vietnamese


In [ ]:
!git clone https://github.com/hungcao0402/PaddleOCR-Vietnamese.git

Cloning into 'PaddleOCR-Vietnamese'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 359 (delta 67), reused 334 (delta 52), pack-reused 0
Receiving objects: 100% (359/359), 12.96 MiB | 34.22 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!pip install -r requirements.txt
# !pip install paddlepaddle-gpu 

In [ ]:
!pip install paddlepaddle-gpu 

In [ ]:
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_r50_vd_sast_icdar15_v2.0_train.tar -P ./pretrain_models
!tar -xf ./pretrain_models/det_r50_vd_sast_icdar15_v2.0_train.tar -C /content/PaddleOCR-Vietnamese/pretrain_models

In [ ]:
!pip install paddlepaddle

In [ ]:
!python3 /content/PaddleOCR-Vietnamese/tools/train.py -c /content/PaddleOCR-Vietnamese/configs/det/SAST.yml

In [ ]:
import json
import os
import re
def no_accent_vietnamese(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s
file1 = open("/content/test_label.txt", "a")
folder = "/content/drive/MyDrive/OCR_anhdangkiem_label/test_label"
for i, filename in enumerate(os.listdir(folder)):
  print(filename)
  json_path = folder + "/" + filename
# json_path = '/content/dkiem0.json'
  file_name = os.path.basename(json_path)
  file_name = file_name[:file_name.find('.')]+'.jpg'
  with open(json_path, 'r') as f:
    data = json.load(f)
    annotations = data[0]['annotations']
    row =str(file_name)+'\t'+ '['
    for i in range(len(annotations)):
      transcription = annotations[i]['label']
      x = annotations[i]['coordinates']['x']
      y = annotations[i]['coordinates']['y']
      width = annotations[i]['coordinates']['width']/2
      height = annotations[i]['coordinates']['height']/2
      coor1 = [round(x-width,2), round(y-height,2)]
      coor2 = [round(x,2), round(y-height,2)]
      coor3 = [round(x,2), round(y,2)]
      coor4 = [round(x-width,2), round(y,2)]
      # print(transcription," ",coor1," ",coor2," ", coor3, ' ', coor4)
      sign = ', '
      if i==len(annotations)-1:
        sign=']'
      row = row+'{"transcription": "'+str(transcription)+'", "points": ['+str(coor1)+', '+str(coor2)+', '+str(coor3)+', '+str(coor4)+']}'+ sign
    file1.write(row+ '\n')
file1.close()

dkiem0.json
dkiem2.json
dkiem3.json
dkiem4.json
dkiem5.json


In [ ]:
import yaml

with open("configs/det/SAST.yml", "r") as f:
    config = yaml.safe_load(f)
config.update({
    'wandb': {
        'project': 'text_detection'
    }
})
config['Global'].update({
    'epoch_num': 5,
    'eval_batch_step': [0, 50],
    'calc_metric_during_train': True
})

with open("configs/det/det_mv3_db.yml", "w") as f:
    yaml.safe_dump(config, f)

In [ ]:
!python3 -m pip install paddlepaddle-gpu==2.4.2 -f https://paddlepaddle.org.cn/whl/stable/noavx.html!

In [ ]:
!pip install paddlepaddle-gpu

In [ ]:
!pip install paddlepaddle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 /content/drive/MyDrive/PaddleOCR-Vietnamese/tools/train.py -c /content/drive/MyDrive/PaddleOCR-Vietnamese/configs/det/SAST.yml

In [ ]:
!nvidia-smi

In [ ]:
!gdown --id 1UUQhNvzgpZy7zXBFQp0Qox-BBjunZ0ml
!unzip vietnamese_original.zip -d train

In [ ]:
import glob 
import os
import numpy as np 
from tqdm import tqdm 
import pandas as pd  
import json

root_path = glob.glob('/content/label/*')
train_label = open("train_label.txt","w")
test_label = open("test_label.txt","w")
for file in root_path:
    with open(file) as f:
      content = f.readlines()
      f.close()
    content = [x.strip() for x in content]
    print(content)
    text = []
    for i in content:
      label = {}
      i = i.split(',',8)
      label['transcription'] = i[-1]
      label['points'] = [[i[0],i[1]],[i[2],i[3]], [i[4],i[5]],[i[6],i[7]]]
      text.append(label)
    content = []
    text = json.dumps(text, ensure_ascii=False)
    img_name = os.path.basename(file).split('.')[0]
    print(img_name)

['dkiem6.jpg\t[{"transcription": "1. PHƯƠNG TIỆN", "points": [[369.1, 212.38], [584.6, 212.38], [584.6, 256.88], [369.1, 256.88]]}, {"transcription": "(VEHICLE)", "points": [[835.1, 230.38], [968.6, 230.38], [968.6, 270.38], [835.1, 270.38]]}, {"transcription": "Biển đăng ký:", "points": [[230.1, 291.38], [361.6, 291.38], [361.6, 317.88], [230.1, 317.88]]}, {"transcription": "83A-087.87", "points": [[499.1, 291.38], [617.1, 291.38], [617.1, 319.88], [499.1, 319.88]]}, {"transcription": "Số quản lý:", "points": [[904.1, 310.38], [1014.1, 310.38], [1014.1, 338.88], [904.1, 338.88]]}, {"transcription": "8301V-017668", "points": [[1126.1, 306.38], [1262.6, 306.38], [1262.6, 342.88], [1126.1, 342.88]]}, {"transcription": "(Registration Number)", "points": [[226.1, 343.38], [417.6, 343.38], [417.6, 370.88], [226.1, 370.88]]}, {"transcription": "(Vehicle Inspection No.)", "points": [[900.1, 363.38], [1108.6, 363.38], [1108.6, 395.38], [900.1, 395.38]]}, {"transcription": "Loại phương tiện: (T

In [ ]:
%cd /content/drive/MyDrive/PaddleOCR-Vietnamese

/content/drive/MyDrive/PaddleOCR-Vietnamese


In [ ]:
import json
import os
import cv2
import copy
import numpy as np
from tools.infer.utility import draw_ocr_box_txt, get_rotate_crop_image

def print_draw_crop_rec_res( img_crop_list, img_name):
        bbox_num = len(img_crop_list)
        for bno in range(bbox_num):
          crop_name=img_name+'_'+str(bno)+'.jpg'
          crop_name_w = "./train/img_crop/{}".format(crop_name)
          cv2.imwrite(crop_name_w, img_crop_list[bno])
          crop_label.write("{0}\t{1}\n".format(crop_name, text[bno]))

crop_label = open('./train/crop_label.txt','w')
with open('./train/labels/train_label.txt','r') as file_text:
  img_files=file_text.readlines()
  
count=0
for img_file in img_files:
  content = json.loads(img_file.split('\t')[1].strip())

  dt_boxes=[]
  text=[]
  
  for i in content:
    content = i['points']
    if i['transcription'] == "###":
      count+=1
      continue
    bb = np.array(i['points'],dtype=np.float32)
    dt_boxes.append(bb)
    text.append(i['transcription'])

  image_file = './train/vietnamese/train_images/' +img_file.split('\t')[0]
  img = cv2.imread(image_file)
  ori_im=img.copy()
  img_crop_list=[]

  for bno in range(len(dt_boxes)):
    tmp_box = copy.deepcopy(dt_boxes[bno])
    img_crop = get_rotate_crop_image(ori_im, tmp_box)
    img_crop_list.append(img_crop)
  img_name = img_file.split('\t')[0].split('.')[0]
  
  if not os.path.exists('./train/img_crop'):
    os.mkdir('./train/img_crop') 
  print_draw_crop_rec_res(img_crop_list,img_name)